In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
from torch import device, cuda

import pandas as pd

In [2]:
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = SentenceTransformer("all-mpnet-base-v2")
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:
reports_file_path = './data/br/netbeans/netbeans.csv'
relations_file_path = 'data/br/netbeans/netbeans_pairs - Copia.csv'

reports = pd.read_csv(reports_file_path, index_col='bug_id')
relations = pd.read_csv(relations_file_path)

In [5]:
relations.head(200)

,issue_id,duplicate
0,200622,185122;191046;192167;168052;201116;179303;1732...
1,186130,187030;186430;186890;187765;189885;186740;1954...
2,8715,NaN
3,47174,NaN
4,89581,NaN
...,...,...
195,197338,NaN
196,85083,83052;82364
197,63541,NaN
198,168032,NaN


In [6]:
train_examples = []

counter_invalid = 0

for index, row in relations.iterrows():

  bug_id = row[0]

  if ( not pd.isna(row[1])):
    duplicate_id = int(row[1].split(";")[0])
    try:
    
      bug_desc = reports.loc[bug_id]['description']
      dup_desc = reports.loc[duplicate_id]['description']

      train_examples.append(InputExample(texts=(
        bug_desc,
        dup_desc
      )))
    
    except Exception as e:
      counter_invalid+=1
      pass
      #print('Unable to reach')

print(len(train_examples[0:500]))

C:\Users\lpros\AppData\Local\Temp\ipykernel_14284\65395072.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bug_id = row[0]
C:\Users\lpros\AppData\Local\Temp\ipykernel_14284\65395072.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( not pd.isna(row[1])):
C:\Users\lpros\AppData\Local\Temp\ipykernel_14284\65395072.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  duplicate_id = int(row[1].split(";")[0])


500


In [7]:
print(len(train_examples))

train_examples = train_examples[0:15000]
print(len(train_examples))

34114
15000


In [8]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=5)

In [9]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [10]:
model.fit(train_objectives=[(train_dataloader, train_loss)])

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3000 [00:00<?, ?it/s]

In [11]:
# Sentences are encoded by calling model.encode()
emb1 = model.encode("Bao is a great dev")
emb2 = model.encode("I know a programmer named Bao")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.6783]])


In [12]:
model.save('./test-15k/')